## ДЗ по поиску

Привет! Вам надо реализивать поисковик на базе вопросов-ответов с сайта [pravoved.ru](https://pravoved.ru/questions-archive/).        
Поиск должен работать на трех технологиях:       
1. обратном индексе     
2. word2vec         
3. doc2vec      

Вы должны понять, какой метод и при каких условиях эксперимента на этом корпусе работает лучше.          
Для измерения качества поиска найдите точность (accuracy) выпадания правильного ответа на конкретный вопрос (в этой базе у каждого вопроса есть только один правильный ответ). Точность нужно измерить для всей базы.    
При этом давайте считать, что выпал правильный ответ, если он попал в **топ-5** поисковой выдачи.

> Сделайте ваш поиск максимально качественным, чтобы значение точности стремилось к 1.     
Для этого можно поэкспериментировать со следующим:       
- модель word2vec (можно брать любую из опен сорса или обучить свою)
- способ получения вектора документа через word2vec: простое среднее арифметическое или взвешивать каждый вектор в соответствии с его tf-idf      
- количество эпох у doc2vec (начинайте от 100)
- предобработка документов для обучения doc2vec (удалять / не удалять стоп-слова)
- блендинг методов поиска: соединить результаты обратного индекса и w2v, или (что проще) w2v и d2v

На это задание отведем 10 дней. Дэдлайн сдачи до полуночи 12.10.

Я правда старался((((((((

In [1]:
import pickle

with open('qa_corpus.pkl', 'rb') as file:
    qa_corpus = pickle.load(file)

Всего в корпусе 1384 пары вопрос-ответ

In [2]:
len(qa_corpus)

1384

Первый элемент блока это вопрос, второй - ответ на него

In [3]:
qa_corpus[0]

['\nДобрый день.Мой сын гражданин Украины (ДНР),имеет вид на жительство в Р.Ф., кот.получил проживая с 2014 г. в Нижегородской области.В 2017г. переехал на постоянное место жительство в г.Ростов.Официально трудоустроился на одно из промышл.предприятий г.Ростова.Оформил временную регистрацию в Ростове.В УФМС предупредили,что по истечении 90 дней он должен либо постоянно прописаться либо покинуть территорию России.Прошу проконсультировать как быть дальше.(Вернуться домой в Донецк,но здесь идет война,работы нет.В Ростове он работает по специальности.Он инженер машиностроитель.)Временная прописка до 15 марта.  Если он сможет приобрести какую либо недвижимость,как долго будет решаться вопрос о его постоянной прописке в Ростове.Как в этом случае будет решаться вопрос с видом на жительство в Ростове? Не получится ли ,что приобретя квартиру,он не успеет в ней прописаться до окончании срока временной регистрации. С уважением Людмила Евгеньевна.\n',
 'Добрый вечер!Из Вашего вопроса вообще ничего

Ну поехали...

##  Word2Vec

Как обычно вначале импортируем все необходимые библиотеки

In [102]:
import gensim
import string
import re
import json
from pymystem3 import Mystem
from gensim import matutils
import numpy as np 
import pandas as pd
from nltk import word_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec, KeyedVectors
from collections import defaultdict
import warnings

warnings.filterwarnings('ignore')
mystem = Mystem()

Теперь берем нашу любимую модель, которая способна на все в нашем мире.

In [109]:
model_path = '/Users/macbook/Downloads/araneum_none_fasttextcbow_300_5_2018/araneum_none_fasttextcbow_300_5_2018.model'
w2v_model = Word2Vec.load(model_path)

Теперь берем готовую функцию обработки текста.

In [110]:
def preprocessing(input_text, del_stopwords=True, del_digit=True):
    """
    :input: raw text
        1. lowercase, del punctuation, tokenize
        2. normal form
        3. del stopwords
        4. del digits
    :return: lemmas
    """
    russian_stopwords = set(stopwords.words('russian'))
    words = [x.lower().strip(string.punctuation+'»«–…') for x in word_tokenize(input_text)]
    lemmas = [mystem.lemmatize(x)[0] for x in words if x]

    lemmas_arr = []
    for lemma in lemmas:
        if del_stopwords:
            if lemma in russian_stopwords:
                continue
        if del_digit:
            if lemma.isdigit():
                continue
        lemmas_arr.append(lemma)
    return lemmas_arr

Сразу напишем нашу функцию для построения вектора

In [112]:
def w2v_vector(word_list):
    some = []
    for word in word_list:
        try:
            some.append(w2v_model.wv[str(word)])
        except KeyError:
            some.append(np.array([0] * 300))
    return matutils.unitvec(np.array(sum(some) / len(word_list)))

Теперь построим словарь, где Ключ – **Вопрос**. Значение – **вектор вопроса** 

In [113]:
%%time

qa_vectors = defaultdict()

for item in qa_corpus:
    qa_vectors[item[1]] = w2v_vector(preprocessing(item[1]))

CPU times: user 15.6 s, sys: 2.96 s, total: 18.5 s
Wall time: 35 s


In [114]:
def similarity(v1, v2):
    return np.dot(v1, v2)

def search(query, number):
    query_vector = w2v_vector(preprocessing(query))
    local_data = defaultdict()
    for item in qa_vectors:
        local_data[item] = similarity(query_vector, qa_vectors[item]) 
    for el in sorted(local_data.items(), key=lambda kv: kv[1], reverse=True)[:number]:
        yield el[0]

Теперь нам необходимо проверить это все в полевых условиях.

In [115]:
%%time

true = 0

for qa in qa_corpus:
    if qa[1] in list(search(qa[0], 5)):
        true += 1

print('For {} attempts, there are(is) {} correct matches (score:{})'.format(len(qa_corpus), true, true / len(qa_corpus)))

For 1384 attempts, there are(is) 368 correct matches (score:0.2658959537572254)
CPU times: user 22.6 s, sys: 1.62 s, total: 24.3 s
Wall time: 33.5 s


Полный конец, давайте сделаем Okapi...

# OkapiBM25

In [43]:
from gensim.summarization import bm25

In [47]:
texts = [preprocessing(x[1]) for x in qa_corpus]
keys = [x[1] for x in qa_corpus]

qa_data = bm25.BM25(texts)

average_idf = sum(map(lambda k: float(qa_data.idf[k]),
                      qa_data.idf.keys())) / len(qa_data.idf.keys())

In [53]:
def okapi25_search(query, number):
    sim = defaultdict()

    for key, value in zip(keys, qa_data.get_scores(query, average_idf)):
        sim[key] = value

    for el in sorted(sim.items(), key=lambda kv: kv[1], reverse=True)[:number]:
        yield el[0]

In [65]:
%%time

true = 0

for qa in qa_corpus:
    if qa[1] in list(okapi25_search(preprocessing(qa[0]), 5)):
        true += 1

print('For {} attempts, there are(is) {} correct matches (score:{})'.format(len(qa_corpus), true, true / len(qa_corpus)))

For 1384 attempts, there are(is) 778 correct matches (score:0.5621387283236994)
CPU times: user 30.7 s, sys: 1.67 s, total: 32.4 s
Wall time: 41.4 s


Aaaaaaaaaany way оно работает, хоть 0.56... Не самый лучший скор... Наверное...

Теперь опробуем...

# Doc2Vec

In [69]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [70]:
def d2v_vector(word_list):
    some = []
    for word in word_list:
        try:
            some.append(d2v_model.wv[str(word)])
        except KeyError:
            some.append(np.array([0] * 300))
    return matutils.unitvec(np.array(sum(some) / len(word_list)))

Поскольку обучаться модель Doc2Vec будет на данных, которые мы подготовим с помощью TaggedDocument, нам необходимо подготовить и почистить текст перед его загрузкой туда. Для этого немного изменим уже существующую функцию.

In [106]:
def d2v_preprocessing(input_text, del_stopwords=True, del_digit=True):
    """
    :input: raw text
        1. lowercase, del punctuation, tokenize
        2. normal form
        3. del stopwords
        4. del digits
    :return: lemmas
    """
    russian_stopwords = set(stopwords.words('russian'))
    words = [x.lower().strip(string.punctuation+'»«–…') for x in word_tokenize(input_text)]
    lemmas = [mystem.lemmatize(x)[0] for x in words if x]

    lemmas_arr = []
    for lemma in lemmas:
        if del_stopwords:
            if lemma in russian_stopwords:
                continue
        if del_digit:
            if lemma.isdigit():
                continue
        lemmas_arr.append(lemma)
    return ' '.join(lemmas_arr)

Теперь получаем наши готовые данные и обучаем нашу модель

In [97]:
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate([d2v_preprocessing(x[1]) for x in qa_corpus])]

In [98]:
%%time

epochs = 100
vec_size = 300
alpha = 0.025

d2v_model = Doc2Vec(size=vec_size,
                    alpha=alpha, 
                    min_alpha=0.00025,
                    min_count=1,
                    dm =1)
  
d2v_model.build_vocab(tagged_data)

print("Vocab created")

d2v_model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=epochs)
print("Model trained")
d2v_model.alpha -= 0.0002
d2v_model.min_alpha = model.alpha

d2v_model.save("d2v_qa.model")
print("Model Saved")

Vocab created
Model trained
Model Saved
CPU times: user 1min 20s, sys: 3.88 s, total: 1min 24s
Wall time: 40.2 s


Получаем вектора и начинаем нашу проверку

In [103]:
%%time

vectors = defaultdict()

for qa in qa_corpus:
    vectors[qa[1]] = d2v_vector(preprocessing(qa[1]))

CPU times: user 15.3 s, sys: 2.91 s, total: 18.2 s
Wall time: 36.5 s


In [104]:
def search(query, result_len):
    query_vector = d2v_vector(preprocessing(query))
    local_data = defaultdict()
    for item in vectors:
        local_data[item] = similarity(query_vector, vectors[item]) 
    sort_data = sorted(local_data.items(), key=lambda kv: kv[1], reverse=True)[:result_len]
    for el in sort_data:
        yield el[0]

def similarity(v1, v2):
    return np.dot(v1, v2)

In [105]:
%%time

true = 0

for qa in qa_corpus:
    if qa[1] in list(search(qa[0], 5)):
        true += 1

print('For {} attempts, there are(is) {} correct matches (score:{})'.format(len(qa_corpus), true, true / len(qa_corpus)))

For 1384 attempts, there are(is) 420 correct matches (score:0.30346820809248554)
CPU times: user 18.8 s, sys: 1.57 s, total: 20.4 s
Wall time: 29.4 s


Теперь попробуем совместить наши модели

## Делаем смузи

Word2Vec + Doc2Vec

In [116]:
def Combine_search(query, result_len):
    w2v_query = w2v_vector(preprocessing(query))
    d2v_query = d2v_vector(preprocessing(query))
    local_data = defaultdict()
    for item in vectors:
        local_data[item] = (similarity(d2v_query, vectors[item])  +
                            similarity(w2v_query, qa_vectors[item])) / 2
    sort_data = sorted(local_data.items(), key=lambda kv: kv[1], reverse=True)[:result_len]
    for el in sort_data:
        yield el[0]

In [117]:
%%time

true = 0

for qa in qa_corpus:
    if qa[1] in list(Combine_search(qa[0], 5)):
        true += 1

print('For {} attempts, there are(is) {} correct matches (score:{})'.format(len(qa_corpus), true, true / len(qa_corpus)))

For 1384 attempts, there are(is) 430 correct matches (score:0.3106936416184971)
CPU times: user 33.8 s, sys: 3.02 s, total: 36.8 s
Wall time: 53.8 s


Как можно заметить, самый большой score мы получили с помощью алгоритма OkapiBM25. Второе место – блэндинг Word2Vec и Doc2Vec. И третье место – Чситый Doc2Vec.